# PIN Estimation

In [1]:
import pandas as pd

from modules import dataset
from modules import pin_estimation

## Load Data

In [3]:
quotes = dataset.load_cedro_quotes()
quotes.sample(5)

,symbol,bucket,open,high,low,close,volume,comprador,indefinido,vendedor,naodireto,direto,rlp
15097,BMOB3,2022-09-08,14.29,14.38,14.12,14.25,301800,730,47,1015,1773,1,18
46556,GPIV33,2023-01-16,2.99,2.99,2.75,2.75,25626,11,0,20,31,0,0
48184,LJQQ3,2023-07-20,5.56,5.61,5.47,5.55,1453500,1947,36,1150,3133,0,0
88943,ONCO3,2022-11-29,7.23,7.41,7.15,7.36,950100,767,22,973,1762,0,0
99935,PGMN3,2023-06-15,3.65,3.90,3.65,3.90,1217400,1273,44,856,2172,1,0


## Estimate PINs

In [4]:
estimate_all_pins = False
if estimate_all_pins:
    pins = pin_estimation.estimate_all_pins(quotes, window=60, verbose=True)
else:
    pins = dataset.load_results()['pin_results']
pins.sample(5)

,symbol,period,pin,alpha,delta,mu,eps_b,eps_s
10408,ESPA3,2022-09-19,0.146343,0.383325,0.521739,2078.833919,2155.903361,2492.460169
51293,PFRM3,2023-05-25,0.098210,0.100001,1.000000,722.851457,331.100053,332.648129
705,TPIS3,2023-07-24,0.237352,0.266666,0.312501,229.267522,85.207586,111.238010
41468,TELB3,2023-05-26,0.419362,0.261454,0.115179,44.563424,3.931111,12.200952
35621,BOAS3,2023-07-17,0.194848,0.366663,0.272729,1267.452318,976.368799,943.982243


# Portfolio Build

In [5]:
from modules import stock_selection
from modules import portfolio_build

In [6]:
eco_data = dataset.load_economatica_quotes()
print(f'Eco-Data type is {type(eco_data)}. Its keys are {eco_data.keys()}')

Eco-Data type is <class 'dict'>. Its keys are dict_keys(['volumes', 'ibov', 'closing_prices'])


In [7]:
eligible_stocks = stock_selection.filter(eco_data['volumes'])
portfolios = portfolio_build.portfolio_build(
    pins,
    eligible_stocks,
)
portfolios

{'pins': {'long':        symbol     period       pin     alpha     delta            mu  \
  4018    SEQL3 2022-06-30  0.193954  0.433354  1.000000   2281.942025   
  57191  AURA33 2022-07-29  0.228066  0.333326  0.699991   3422.926164   
  13534   JHSF3 2022-08-31  0.178792  0.449996  0.777770   2332.391765   
  8483    CASH3 2022-09-30  0.236780  0.083325  0.799966  37872.301500   
  22143   OPCT3 2022-09-30  0.220967  0.416672  0.760001    867.807652   
  ...       ...        ...       ...       ...       ...           ...   
  59185   IRBR3 2023-08-28  0.168738  0.383328  0.956515   3764.882521   
  59407   CGRA3 2023-08-28  0.325164  0.300005  1.000000     45.973807   
  61627   COCE5 2023-08-28  0.209680  0.350504  0.945526     54.906982   
  76149   SEER3 2023-08-28  0.174740  0.333330  0.750000   1454.902737   
  77004   RAIZ4 2023-08-28  0.273132  0.533351  0.781247   9270.317291   
  
               eps_b        eps_s  
  4018   2328.535814  1781.142529  
  57191  2059.777061 

## Calculate Returns

In [8]:
from modules import returns

In [9]:
r = returns.calculate_all_portfolios_returns(portfolios['weights'], eco_data['closing_prices'])
r

,long,short,longonly,long_short
2022-07-01,-0.048870,0.003752,-0.049360,-0.052622
2022-07-04,-0.039894,-0.018692,-0.040385,-0.021203
2022-07-05,0.042575,0.011429,0.042084,0.031146
2022-07-06,0.000490,0.008475,0.000000,-0.007984
2022-07-07,0.004337,0.010271,0.003846,-0.005934
...,...,...,...,...
2023-09-01,0.014920,0.026360,0.014429,-0.011441
2023-09-04,-0.005777,-0.018228,-0.006267,0.012452
2023-09-05,-0.011019,0.029056,-0.011509,-0.040075
2023-09-06,-0.004292,-0.015351,-0.004783,0.011059
